In [2]:
import jax
from jax import grad
import jax.numpy as jnp
import numpy as np

# JAX

### Gradient

In [3]:
alpha=1/2
def f(x):
    return x**alpha

In [ ]:
print(grad(f)(1.))

In [5]:
def test_numpy(A):
    return A@A

def test_jax(A):
    return A@A

@jax.jit
def test_jax_jit(A):
    return A@A    

size=(1000,1000)
A_numpy = np.random.uniform(size=size)
A_jax = jnp.array(A_numpy)

In [6]:
%%timeit
test_numpy(A_numpy)

1.1 s ± 86.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
test_jax(A_jax)

381 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
test_jax_jit(A_jax)

372 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### While Loop

In [10]:
jax.lax.while_loop(lambda x: x[1]<4, lambda x: (x[0], x[1]+1), (-3,-1))

(DeviceArray(-3, dtype=int32, weak_type=True),
 DeviceArray(4, dtype=int32, weak_type=True))